In [1]:
import spacy

In [107]:
nlp = spacy.load('en_core_web_lg', disable=["parser", "tagger", "ner"])

In [108]:
vocab = [w for w in nlp.vocab if w.is_lower and w.prob >= -15 and w.vector.any()]
def most_similar(word, top=10):
    by_similarity = sorted(vocab, key=lambda w: word.similarity(w), reverse=True)
    return by_similarity[:top]

In [109]:
[w.lower_ for w in most_similar(nlp.vocab['computer'])]

['computer',
 'computers',
 'laptop',
 'software',
 'desktop',
 'computing',
 'laptops',
 'workstation',
 'hardware',
 'systems']

In [203]:
import pandas as pd
import numpy as np
pd.set_option('display.max_colwidth', -1)
nlp.max_length = 100000000

In [232]:
fun_fact_df = pd.read_csv('../data/fun_fact_title.csv')

/Users/danny/Documents/College-CORNELL/04 Senior-MEng/Spring 2019/INFO4300 - Language and Information/final project/fun-fax/venv/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (4,8,10,11,13,14,15,16,17,19,20,21,22,23,24,25,26,28,29,30,31,33,37,38,43,44,47,48,49,50,51,52,54,55,57,58,60,61,62,64,65,71,75,87,89,91,92,93,94,97,101,104,105,106,107,110,111,112,113,114,115,118,119,120,121,122,129,133,135,137,138,139,140,142,143,148,152,155,174,176,178,179,180,181,187,191,192,193,194,197,199,201,202,203,209) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [233]:
fun_fact_df = fun_fact_df.dropna(axis='rows', subset=['title'])

In [234]:
concat_docs = " ||| ".join(list(fun_fact_df["title"]))

In [235]:
doc = nlp(concat_docs)

In [236]:
def compute_embedding(tokens):
    return np.average([t.vector for t in tokens], axis=0)

In [237]:
doc_vectors = []
cur_sent = []
for w in doc:
    if w.text == "|||":
        doc_vectors.append(compute_embedding(cur_sent))
        cur_sent = []
        continue
    cur_sent.append(w)
doc_vectors.append(compute_embedding(cur_sent))
doc_vectors = np.array(doc_vectors)

In [238]:
query = "computer"
query_embedding = compute_embedding([w for w in nlp(query)])

In [239]:
rankings = np.dot(doc_vectors, query_embedding)
fun_fact_df["title"][np.argsort(-rankings)]

8264     COMPUTER FUN FACT                                                                                                                                                                                                                                                                                  
8484     Fun fact ...about Windows Phone users                                                                                                                                                                                                                                                              
11487    Mobile App Fun Fact                                                                                                                                                                                                                                                                                
5618     Any thoughts on this computer fun fact?                                                 

In [219]:
doc_vectors.shape

(324996, 300)